In [145]:
# Librerías
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from astroquery.mpc import MPC

# Cobs API

In [146]:
# Verificar la conexión a internet.
def verificar_conexion():
    try:
        requests.get("http://www.google.com", timeout=5)
        print('✅ Conectado a internet.')
        return True
    
    except requests.ConnectionError:
        print('🛑 Sin conexión a internet.')
        return False

In [147]:
# Conexión con la API de COBS
try:
    content = [] 
    fecha_inicial = '1976-01-01 00:00'
    nombre_cometa = '1P'
    Link_cops_API_pagina_1 = f'https://cobs.si/api/obs_list.api?des={nombre_cometa}&format=json&from_date={fecha_inicial}&page=1&exclude_faint=False&exclude_not_accurate=False'

    if True:
        print(f'\n⌛ Conectando con la base de datos [COBS Observaciones].')
        response_pagina_1 = requests.get(Link_cops_API_pagina_1)

        if response_pagina_1.status_code == 200:
            content_pagina_1 = response_pagina_1.json()
            numero_de_paginas = int(content_pagina_1['info']['pages'])

            content.extend(content_pagina_1['objects'])

            for pagina in range(2, numero_de_paginas + 1):
                Link_cops_API_pagina = f'https://cobs.si/api/obs_list.api?des={nombre_cometa}&format=json&from_date={fecha_inicial}&page={pagina}&exclude_faint=False&exclude_not_accurate=False'
                response_pagina = requests.get(Link_cops_API_pagina)
                content_pagina = response_pagina.json()
                content.extend(content_pagina['objects'])

            print('✅ Base de datos actualizada [COBS Observaciones].')
        
except requests.ConnectionError:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response_pagina.status_code}\n{response_pagina.content}')


⌛ Conectando con la base de datos [COBS Observaciones].
✅ Base de datos actualizada [COBS Observaciones].


In [148]:
# Creación del data frame Cometa
cometa_df = pd.DataFrame(content)
cometa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6635 entries, 0 to 6634
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       6635 non-null   object 
 1   obs_date                   6635 non-null   object 
 2   comet                      6635 non-null   object 
 3   observer                   6635 non-null   object 
 4   location                   71 non-null     object 
 5   extinction                 148 non-null    object 
 6   obs_method                 6635 non-null   object 
 7   comet_visibility           19 non-null     object 
 8   magnitude                  6635 non-null   object 
 9   conditions                 253 non-null    object 
 10  ref_catalog                6635 non-null   object 
 11  instrument_aperture        6634 non-null   object 
 12  instrument_type            6634 non-null   object 
 13  instrument_focal_ratio     1898 non-null   float

In [149]:
# Numero de registros y variables sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 6635
Variables: 47


In [150]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
5618,V,1985-11-12 01:55:12,"{'type': 'P', 'name': '1P', 'fullname': '1P/Ha...","{'first_name': 'Christian', 'last_name': 'Glow...",None,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,7.2,None,...,None,None,None,None,None,,None,NaN,,2022-05-08 10:58:38
3657,V,1986-01-06 21:36:00,"{'type': 'P', 'name': '1P', 'fullname': '1P/Ha...","{'first_name': 'Rodrigo', 'last_name': 'Losada...",None,None,"{'key': 'B', 'name': 'Simple Out-Out method', ...",None,5.5,None,...,None,None,None,None,None,,None,NaN,,2022-05-08 10:56:45
2811,V,1986-03-05 18:57:36,"{'type': 'P', 'name': '1P', 'fullname': '1P/Ha...","{'first_name': 'T. B.', 'last_name': 'Tregaski...",None,None,"{'key': 'B', 'name': 'Simple Out-Out method', ...",None,3.1,None,...,None,None,None,None,None,,None,NaN,,2022-05-08 10:55:53
5123,V,1985-11-29 17:31:12,"{'type': 'P', 'name': '1P', 'fullname': '1P/Ha...","{'first_name': 'Attila', 'last_name': 'Kosa-Ki...",None,None,"{'key': 'M', 'name': 'Modified-Out method', 's...",None,5.7,None,...,None,None,None,None,None,,None,NaN,,2022-05-08 10:58:06
5009,V,1985-12-02 18:28:48,"{'type': 'P', 'name': '1P', 'fullname': '1P/Ha...","{'first_name': 'Frriedrich', 'last_name': 'Ger...",None,None,"{'key': '-', 'name': 'Unknown', 'source': None}",None,5.3,None,...,None,None,None,None,None,,None,NaN,,2022-05-08 10:57:59


In [151]:
# Métodos de observación
cometa_df.obs_method.apply(
    lambda registro: f"{registro['key']}: {registro['name']}" if (registro is not None) and ('key' in registro) and ('name' in registro) else 'Datos faltantes'
).value_counts()

obs_method
S: In-Out method                                                 2639
B: Simple Out-Out method                                         1998
M: Modified-Out method                                           1361
-: Unknown                                                        222
G: CCD magnitude with a Corion NR-400 "minus-infrared" filter     153
I: In-focus                                                        89
V: Johnson/Bessel/Kron/Cousins V with CCD                          64
K: Clear filter used on SOHO spacecraft with C3 coronagraph        57
O: Out-of-focus (or extrafocal) method                             20
E: Extrafocal-Extinction method                                    20
N: Called magnitude of nucleus or condensation                      5
g: CCD magnitude with Gunn g filter                                 2
P: photographic                                                     2
R: Photoelectric R                                                  1
C: Unfilt

In [152]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'] if registro is not None and 'key' in registro else 'Dato faltante')
cometa_df['obs_date'] = pd.to_datetime(pd.to_datetime(cometa_df.obs_date).dt.date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [153]:
# Creación del data frame curva de luz cruda
curva_de_luz_cruda_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']].copy()
curva_de_luz_cruda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6635 entries, 0 to 6634
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  6635 non-null   object        
 1   obs_date        6635 non-null   datetime64[ns]
 2   magnitude       6635 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 155.6+ KB


In [154]:
# Numero de registros y variables con la información filtrada
filas,columnas = curva_de_luz_cruda_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 6635
Variables: 3


In [155]:
# Data Frame de la curva de luz
curva_de_luz_cruda_df.sample(5)

,obs_method_key,obs_date,magnitude
2551,G,1986-03-15,2.8
2711,M,1986-03-09,3.5
6537,M,1985-08-26,14.6
3284,B,1986-01-17,4.2
36,I,1987-11-29,13.5


In [156]:
# Curva de luz cruda.
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_cruda_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

# Perihelio Cobs API

In [157]:
# Conexión con la API de COBS para obtener el perihelio
try: 
    Link_cops_API = f'https://cobs.si/api/comet.api?des={nombre_cometa}'

    if verificar_conexion():
        response = requests.get(Link_cops_API)

        if response.status_code == 200:
            perihelio = pd.to_datetime(response.json()['object']['perihelion_date'])
            print('✅ Perihelio del cometa obtenido.')
    
except requests.ConnectionError:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Conectado a internet.
✅ Perihelio del cometa obtenido.


# MPC API usando astroquery.

In [ ]:
# Creación de data frame Ephemeris (conexión con la API del MPC)
fecha_inicial = curva_de_luz_cruda_df.obs_date.min()
fecha_final = curva_de_luz_cruda_df.obs_date.max()
dias_entre_fechas = (fecha_final - fecha_inicial).days + 1

fechas = dias_entre_fechas + 1 if dias_entre_fechas <= 1441 else 1441

ephemeris = MPC.get_ephemeris(nombre_cometa, start = str(fecha_inicial), number = fechas)  # type: ignore

ephemeris_df = ephemeris.to_pandas()
ephemeris_df.columns = ephemeris_df.columns.str.lower().str.replace(' ', '_')
ephemeris_df

,date,ra,dec,delta,r,elongation,phase,v,proper_motion,direction
0,1981-12-02,108.025000,8.389444,12.023,12.789,139.5,2.9,19.7,10.82,268.3
1,1981-12-03,107.951667,8.387500,12.007,12.783,140.5,2.8,19.7,10.97,268.6
2,1981-12-04,107.877500,8.385833,11.992,12.778,141.5,2.7,19.7,11.12,268.9
3,1981-12-05,107.802083,8.384722,11.977,12.773,142.5,2.7,19.7,11.27,269.2
4,1981-12-06,107.725417,8.383889,11.962,12.768,143.5,2.6,19.7,11.41,269.5
...,...,...,...,...,...,...,...,...,...,...
1436,1985-11-07,71.751667,23.040278,0.853,1.787,151.5,15.4,7.4,215.65,271.9
1437,1985-11-08,70.139583,23.080556,0.829,1.773,153.9,14.2,7.3,229.61,271.2
1438,1985-11-09,68.422917,23.103889,0.805,1.758,156.5,13.0,7.2,244.27,270.5
1439,1985-11-10,66.597083,23.106667,0.783,1.744,159.1,11.7,7.1,259.59,269.7


In [197]:
# Info del data frame ephemeris
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1441 entries, 0 to 1440
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1441 non-null   datetime64[ns]
 1   ra             1441 non-null   float64       
 2   dec            1441 non-null   float64       
 3   delta          1441 non-null   float64       
 4   r              1441 non-null   float64       
 5   elongation     1441 non-null   float64       
 6   phase          1441 non-null   float64       
 7   v              1441 non-null   float64       
 8   proper_motion  1441 non-null   float64       
 9   direction      1441 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 112.7 KB


In [160]:
# Dar a los datos el formato deseado
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = pd.to_datetime(ephemeris_df.date.dt.date)
ephemeris_df.dtypes

date             datetime64[ns]
ra                      float64
dec                     float64
delta                   float64
r                       float64
elongation              float64
phase                   float64
v                       float64
proper_motion           float64
direction               float64
dtype: object

In [161]:
# Creación del data frame ephemeris filtrada
ephemeris_filtrada_df = ephemeris_df[['date', 'delta','r', 'phase']].copy()
ephemeris_filtrada_df = ephemeris_filtrada_df.rename(columns = {'date':'obs_date'})
ephemeris_filtrada_df

,obs_date,delta,r,phase
0,1981-12-02,12.023,12.789,2.9
1,1981-12-03,12.007,12.783,2.8
2,1981-12-04,11.992,12.778,2.7
3,1981-12-05,11.977,12.773,2.7
4,1981-12-06,11.962,12.768,2.6
...,...,...,...,...
1436,1985-11-07,0.853,1.787,15.4
1437,1985-11-08,0.829,1.773,14.2
1438,1985-11-09,0.805,1.758,13.0
1439,1985-11-10,0.783,1.744,11.7


# Unión de las bases de datos.

In [162]:
# Unión de las bases de datos COBS y MPC
curva_de_luz_procesada_df = curva_de_luz_cruda_df.merge(ephemeris_filtrada_df, on='obs_date')
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase
0,B,1985-11-11,8.4,0.761,1.730,10.3
1,B,1985-11-11,7.6,0.761,1.730,10.3
2,M,1985-11-11,7.7,0.761,1.730,10.3
3,B,1985-11-11,8.0,0.761,1.730,10.3
4,B,1985-11-11,8.6,0.761,1.730,10.3
...,...,...,...,...,...,...
1007,V,1984-01-27,23.7,7.160,7.988,4.0
1008,V,1983-10-10,23.2,8.689,8.754,6.6
1009,V,1983-01-14,23.5,9.546,10.484,1.7
1010,V,1982-10-17,24.0,10.885,11.016,5.2


In [163]:
# Información del data frame curva de lus procesada
curva_de_luz_procesada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1012 entries, 0 to 1011
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  1012 non-null   object        
 1   obs_date        1012 non-null   datetime64[ns]
 2   magnitude       1012 non-null   float64       
 3   delta           1012 non-null   float64       
 4   r               1012 non-null   float64       
 5   phase           1012 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 47.6+ KB


In [164]:
# Reducción de la magnitud aparente y calculo del Delta t
beta = 0

curva_de_luz_procesada_df['magnitud_reducida'] = (
    curva_de_luz_cruda_df['magnitude'] 
    - 5 * np.log10(curva_de_luz_procesada_df['delta'] * curva_de_luz_procesada_df['r'])
    - (beta * curva_de_luz_procesada_df['phase'])
    )

curva_de_luz_procesada_df['delta_t'] = (curva_de_luz_procesada_df.obs_date - perihelio) # type: ignore
curva_de_luz_procesada_df['delta_t'] = curva_de_luz_procesada_df.delta_t.apply(lambda delta_t: delta_t.days)

curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,B,1985-11-11,8.4,0.761,1.730,10.3,21.402846,-27695
1,B,1985-11-11,7.6,0.761,1.730,10.3,18.702846,-27695
2,M,1985-11-11,7.7,0.761,1.730,10.3,18.502846,-27695
3,B,1985-11-11,8.0,0.761,1.730,10.3,18.502846,-27695
4,B,1985-11-11,8.6,0.761,1.730,10.3,18.502846,-27695
...,...,...,...,...,...,...,...,...
1007,V,1984-01-27,23.7,7.160,7.988,4.0,-3.186755,-28349
1008,V,1983-10-10,23.2,8.689,8.754,6.6,-3.705882,-28458
1009,V,1983-01-14,23.5,9.546,10.484,1.7,-4.901742,-28727
1010,V,1982-10-17,24.0,10.885,11.016,5.2,-4.794262,-28816


In [165]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='obs_date', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [166]:
# Curva de luz reducida
labels = {'delta_t':'t-Δt','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='delta_t', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [167]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_promediada_df = curva_de_luz_procesada_df.copy()
curva_de_luz_promediada_df['promedio_movil'] = curva_de_luz_promediada_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_promediada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,B,1985-11-11,8.4,0.761,1.730,10.3,21.402846,-27695,NaN
1,B,1985-11-11,7.6,0.761,1.730,10.3,18.702846,-27695,NaN
2,M,1985-11-11,7.7,0.761,1.730,10.3,18.502846,-27695,NaN
3,B,1985-11-11,8.0,0.761,1.730,10.3,18.502846,-27695,NaN
4,B,1985-11-11,8.6,0.761,1.730,10.3,18.502846,-27695,NaN
...,...,...,...,...,...,...,...,...,...
1007,V,1984-01-27,23.7,7.160,7.988,4.0,-3.186755,-28349,-3.257257
1008,V,1983-10-10,23.2,8.689,8.754,6.6,-3.705882,-28458,-3.288746
1009,V,1983-01-14,23.5,9.546,10.484,1.7,-4.901742,-28727,-3.605358
1010,V,1982-10-17,24.0,10.885,11.016,5.2,-4.794262,-28816,-3.763759


In [168]:
# Curva de luz Promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='obs_date', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [169]:
# Curva de luz Promediada
labels = {'delta_t':'t - Δt','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='delta_t', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v1 (Promedio corrido -> agrupación)

In [170]:
# Creación del data frame curva de luz agrupada
curva_de_luz_interna_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').max()
curva_de_luz_interna_v1_df = curva_de_luz_interna_v1_df.reset_index()

curva_de_luz_interna_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,1981-12-02,V,24.7,12.023,12.789,2.9,-5.834247,-29135,-4.127872
1,1982-10-17,V,24.0,10.885,11.016,5.2,-4.794262,-28816,-3.763759
2,1983-01-14,V,23.5,9.546,10.484,1.7,-4.901742,-28727,-3.605358
3,1983-10-10,V,23.2,8.689,8.754,6.6,-3.705882,-28458,-3.288746
4,1984-01-27,V,23.7,7.160,7.988,4.0,-3.186755,-28349,-3.257072
...,...,...,...,...,...,...,...,...,...
96,1985-11-07,S,8.8,0.853,1.787,15.4,12.384632,-27699,11.598918
97,1985-11-08,S,12.0,0.829,1.773,14.2,12.863684,-27698,12.010679
98,1985-11-09,S,9.7,0.805,1.758,13.0,13.245926,-27697,12.369526
99,1985-11-10,S,9.1,0.783,1.744,11.7,13.223459,-27696,12.837745


In [171]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='red', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [172]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v1_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='red', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v1 (Promedio corrido -> agrupación)

In [173]:
# Creación del data frame curva de luz agrupada
curva_de_luz_externa_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').min()
curva_de_luz_externa_v1_df = curva_de_luz_externa_v1_df.reset_index()
curva_de_luz_externa_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,1981-12-02,V,24.7,12.023,12.789,2.9,-5.834247,-29135,-4.127872
1,1982-10-17,V,24.0,10.885,11.016,5.2,-4.794262,-28816,-3.763759
2,1983-01-14,V,23.5,9.546,10.484,1.7,-4.901742,-28727,-3.605358
3,1983-10-10,V,23.2,8.689,8.754,6.6,-3.705882,-28458,-3.288746
4,1984-01-27,V,23.4,7.160,7.988,4.0,-3.386755,-28349,-3.257257
...,...,...,...,...,...,...,...,...,...
96,1985-11-07,-,6.4,0.853,1.787,15.4,10.084632,-27699,10.970346
97,1985-11-08,-,6.3,0.829,1.773,14.2,11.163684,-27698,11.435112
98,1985-11-09,B,6.0,0.805,1.758,13.0,11.045926,-27697,11.860212
99,1985-11-10,-,6.2,0.783,1.744,11.7,11.823459,-27696,12.352030


In [174]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='yellow', size=6, line= dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [175]:
# Gráfica de lus promediada
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v1_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='yellow', size=6, line= dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v2 (Agrupación  -> promedio corrido)

In [176]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_max_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').max()
curva_de_luz_agrupada_max_v2_df = curva_de_luz_agrupada_max_v2_df.reset_index()
curva_de_luz_agrupada_max_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,1981-12-02,V,24.7,12.023,12.789,2.9,-5.834247,-29135
1,1982-10-17,V,24.0,10.885,11.016,5.2,-4.794262,-28816
2,1983-01-14,V,23.5,9.546,10.484,1.7,-4.901742,-28727
3,1983-10-10,V,23.2,8.689,8.754,6.6,-3.705882,-28458
4,1984-01-27,V,23.7,7.160,7.988,4.0,-3.186755,-28349
...,...,...,...,...,...,...,...,...
96,1985-11-07,S,8.8,0.853,1.787,15.4,12.384632,-27699
97,1985-11-08,S,12.0,0.829,1.773,14.2,12.863684,-27698
98,1985-11-09,S,9.7,0.805,1.758,13.0,13.245926,-27697
99,1985-11-10,S,9.1,0.783,1.744,11.7,13.223459,-27696


In [177]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_max_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [178]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_interna_v2_df = curva_de_luz_agrupada_max_v2_df.copy()
curva_de_luz_interna_v2_df['promedio_movil'] = curva_de_luz_interna_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_interna_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,1981-12-02,V,24.7,12.023,12.789,2.9,-5.834247,-29135,NaN
1,1982-10-17,V,24.0,10.885,11.016,5.2,-4.794262,-28816,NaN
2,1983-01-14,V,23.5,9.546,10.484,1.7,-4.901742,-28727,NaN
3,1983-10-10,V,23.2,8.689,8.754,6.6,-3.705882,-28458,NaN
4,1984-01-27,V,23.7,7.160,7.988,4.0,-3.186755,-28349,NaN
...,...,...,...,...,...,...,...,...,...
96,1985-11-07,S,8.8,0.853,1.787,15.4,12.384632,-27699,9.562190
97,1985-11-08,S,12.0,0.829,1.773,14.2,12.863684,-27698,10.426494
98,1985-11-09,S,9.7,0.805,1.758,13.0,13.245926,-27697,11.034364
99,1985-11-10,S,9.1,0.783,1.744,11.7,13.223459,-27696,11.656443


In [179]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='red', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [180]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v2_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='red', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v2 (Agrupación  -> promedio corrido)

In [181]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_min_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').min()
curva_de_luz_agrupada_min_v2_df = curva_de_luz_agrupada_min_v2_df.reset_index()
curva_de_luz_agrupada_min_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,1981-12-02,V,24.7,12.023,12.789,2.9,-5.834247,-29135
1,1982-10-17,V,24.0,10.885,11.016,5.2,-4.794262,-28816
2,1983-01-14,V,23.5,9.546,10.484,1.7,-4.901742,-28727
3,1983-10-10,V,23.2,8.689,8.754,6.6,-3.705882,-28458
4,1984-01-27,V,23.4,7.160,7.988,4.0,-3.386755,-28349
...,...,...,...,...,...,...,...,...
96,1985-11-07,-,6.4,0.853,1.787,15.4,10.084632,-27699
97,1985-11-08,-,6.3,0.829,1.773,14.2,11.163684,-27698
98,1985-11-09,B,6.0,0.805,1.758,13.0,11.045926,-27697
99,1985-11-10,-,6.2,0.783,1.744,11.7,11.823459,-27696


In [182]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_min_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa}')
fig.update_yaxes(autorange="reversed")
fig.show()

In [183]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_externa_v2_df = curva_de_luz_agrupada_min_v2_df.copy()
curva_de_luz_externa_v2_df['promedio_movil'] = curva_de_luz_externa_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_externa_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,1981-12-02,V,24.7,12.023,12.789,2.9,-5.834247,-29135,NaN
1,1982-10-17,V,24.0,10.885,11.016,5.2,-4.794262,-28816,NaN
2,1983-01-14,V,23.5,9.546,10.484,1.7,-4.901742,-28727,NaN
3,1983-10-10,V,23.2,8.689,8.754,6.6,-3.705882,-28458,NaN
4,1984-01-27,V,23.4,7.160,7.988,4.0,-3.386755,-28349,NaN
...,...,...,...,...,...,...,...,...,...
96,1985-11-07,-,6.4,0.853,1.787,15.4,10.084632,-27699,7.319333
97,1985-11-08,-,6.3,0.829,1.773,14.2,11.163684,-27698,7.940780
98,1985-11-09,B,6.0,0.805,1.758,13.0,11.045926,-27697,8.620078
99,1985-11-10,-,6.2,0.783,1.744,11.7,11.823459,-27696,9.385015


In [184]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='yellow', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [185]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v2_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='yellow', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz usando la mediada v1 (Mediana de cada día registrado).

In [186]:
# Creación del data frame curva de luz mediana v1
curva_de_luz_mediada_v1_df = curva_de_luz_procesada_df.groupby(by= 'obs_date').median(numeric_only= True)
curva_de_luz_mediada_v1_df = curva_de_luz_mediada_v1_df.reset_index()
curva_de_luz_mediada_v1_df

,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,1981-12-02,24.70,12.023,12.789,2.9,-5.834247,-29135.0
1,1982-10-17,24.00,10.885,11.016,5.2,-4.794262,-28816.0
2,1983-01-14,23.50,9.546,10.484,1.7,-4.901742,-28727.0
3,1983-10-10,23.20,8.689,8.754,6.6,-3.705882,-28458.0
4,1984-01-27,23.55,7.160,7.988,4.0,-3.286755,-28349.0
...,...,...,...,...,...,...,...
96,1985-11-07,7.90,0.853,1.787,15.4,11.084632,-27699.0
97,1985-11-08,7.95,0.829,1.773,14.2,11.563684,-27698.0
98,1985-11-09,7.50,0.805,1.758,13.0,11.945926,-27697.0
99,1985-11-10,7.25,0.783,1.744,11.7,12.723459,-27696.0


In [187]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v1_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [188]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v1_df, x='delta_t', y='magnitud_reducida', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz usando la mediada v2 (Mediana de las dos curvas).

In [189]:
# Creación del data frame curva de luz mediana v2
curva_de_luz_mediada_v2_df = curva_de_luz_externa_v2_df.copy()
curva_de_luz_mediada_v2_df['mediana'] = (curva_de_luz_interna_v2_df['promedio_movil'] + curva_de_luz_externa_v2_df['promedio_movil'])/2
curva_de_luz_mediada_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil,mediana
0,1981-12-02,V,24.7,12.023,12.789,2.9,-5.834247,-29135,NaN,NaN
1,1982-10-17,V,24.0,10.885,11.016,5.2,-4.794262,-28816,NaN,NaN
2,1983-01-14,V,23.5,9.546,10.484,1.7,-4.901742,-28727,NaN,NaN
3,1983-10-10,V,23.2,8.689,8.754,6.6,-3.705882,-28458,NaN,NaN
4,1984-01-27,V,23.4,7.160,7.988,4.0,-3.386755,-28349,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
96,1985-11-07,-,6.4,0.853,1.787,15.4,10.084632,-27699,7.319333,8.440761
97,1985-11-08,-,6.3,0.829,1.773,14.2,11.163684,-27698,7.940780,9.183637
98,1985-11-09,B,6.0,0.805,1.758,13.0,11.045926,-27697,8.620078,9.827221
99,1985-11-10,-,6.2,0.783,1.744,11.7,11.823459,-27696,9.385015,10.520729


In [190]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v2_df, x='obs_date', y='mediana', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [191]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v2_df, x='delta_t', y='mediana', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa}')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Comparación de las curvas de luz v1 (Promedio corrido -> agrupación)

In [192]:
# Gráfica de luz promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v1_df.obs_date, y=curva_de_luz_externa_v1_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.obs_date, y=curva_de_luz_interna_v1_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.obs_date, y=curva_de_luz_mediada_v1_df.magnitud_reducida, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.obs_date, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente_v2', marker=dict(color='green', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.obs_date, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo_v2', marker=dict(color='blue', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.obs_date, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana_v2', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()

In [193]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v1_df.delta_t, y=curva_de_luz_externa_v1_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.delta_t, y=curva_de_luz_interna_v1_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.delta_t, y=curva_de_luz_mediada_v1_df.magnitud_reducida, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='t - Δt', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()

# Comparación de las curvas de luz v2 (Agrupación -> promedio corrido)

In [194]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.obs_date, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.obs_date, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.obs_date, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()

In [195]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.delta_t, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.delta_t, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.delta_t, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='t - Δt', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa}')
fig.show()